In [1]:
import operator
import os
import random
import functools
import collections
import joblib
import random
import nltk
import sklearn
import sklearn_crfsuite
import numpy as np
from itertools import chain
from nltk.corpus import PlaintextCorpusReader 
from nltk import sent_tokenize, word_tokenize, pos_tag 
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import KFold

random.seed(1999)

In [2]:
#função do tamanho da palavra (returna True se for maior que 4)
def length(word):
    if len(word) >= 4: 
        tamanho = True
    else:
        tamanho = False
    return tamanho

teste_tagger = joblib.load('POS_tagger_brill.pkl')

def postag(word):
    phrase = word
    postag = teste_tagger.tag(word_tokenize(phrase))
    return postag[0][1]

#tamanho da setenca
def tamsent(sent,i):
    conta = []
    valor = []
    for i in range(len(sent)):
        conta.append(sent[i].count(sent[i][0]))
    valor = sum(conta)
    return valor

#frequencia da palavra na sentenca
def freqwordsent(sent,word):
    conta = []
    valor = []
    for j in range(len(sent)):
        conta.append(sent[j].count(word))
    valor = sum(conta)
    return valor


def word2features(sent, i):
    word = sent[i][0]  
    features = {
        'bias': 1.0,
        'word': word,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[-1:]': word[-1:],        
        'word[:1]': word[:1],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag(word),
        'postag[:2]': postag(word)[:1],
        'postag[:2]': postag(word)[:2],
        'tamanho': length(word),
        'word.isalnum()' : word.isalnum(),
        'len(word)': len(word),
        'tamanho(sent)': tamsent(sent,i),
        'freqwordsent' : freqwordsent(sent,word),   
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word': word1,
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag(word1),
            '-1:postag[:2]': postag(word1)[:1],
            '-1:postag[:2]': postag(word1)[:2],
            '-1:word[-3:]': word1[-3:],
            '-1:word[-2:]': word1[-2:],
            '-1:word[-1:]': word1[-1:],
            '-1:word[:1]': word1[:1],
            '-1:word[:2]': word1[:2],
            '-1:word[:3]': word1[:3],
            '-1:len(word)': len(word1),
            '-1:word.isalnum()' : word1.isalnum(),
        })
    else:
        features['Inicio'] = True

    if i > 1:
        word2 = sent[i-2][0]
        features.update({
            '-2:word': word2,
            '-2:word.lower()': word2.lower(),
            '-2:word.istitle()': word2.istitle(),
            '-2:word.isdigit()': word2.isdigit(),
            '-2:word.isupper()': word2.isupper(),
            '-2:postag': postag(word2),
            '-2:postag[:2]': postag(word2)[:2],
            '-2:word[-3:]': word2[-3:],
            '-2:word[-2:]': word2[-2:],
            '-2:word[-1:]': word2[-1:],
            '-2:word[:1]': word2[:1],
            '-2:word[:2]': word2[:2],
            '-2:word[:3]': word2[:3],
            '-2:len(word)': len(word2),
            '-2:word.isalnum()' : word2.isalnum(),

        })
    if i < len(sent)-1:
        word3 = sent[i+1][0]
        features.update({
            '+1:word': word3,
            '+1:word.lower()': word3.lower(),
            '+1:word.istitle()': word3.istitle(),
            '+1:word.isdigit()': word3.isdigit(),
            '+1:word.isupper()': word3.isupper(),
            '+1:postag': postag(word3),
            '+1:postag[:2]': postag(word3)[:2],
            '+1:word[-3:]': word3[-3:],
            '+1:word[-2:]': word3[-2:],
            '+1:word[-1:]': word3[-1:],
            '+1:word[:1]': word3[:1],
            '+1:word[:2]': word3[:2],
            '+1:word[:3]': word3[:3],
            '+1:len(word)': len(word3),
            '+1:word.isalnum()' : word3.isalnum()
            })
    else:
        features['Final'] = True
   
    if i < len(sent)-2:
        word4 = sent[i+2][0]
        features.update({
            '+2:word': word4,
            '+2:word.lower()': word4.lower(),
            '+2:word.istitle()': word4.istitle(),
            '+2:word.isdigit()': word4.isdigit(),
            '+2:word.isupper()': word4.isupper(),
            '+2:postag': postag(word4),
            '+2:postag[:2]': postag(word4)[:2],
            '+2:word[-3:]': word4[-3:],
            '+2:word[-2:]': word4[-2:],
            '+2:word[-1:]': word4[-1:],
            '+2:word[:1]': word4[:1],
            '+2:word[:2]': word4[:2],
            '+2:word[:3]': word4[:3],
            '+2:len(word)': len(word4),
            '+2:word.isalnum()' : word4.isalnum()
        })     

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

lista = [
        'bias',
        'word',
        'word.lower()',
        'word[-3:]',
        'word[-2:]',
        'word[-1:]',
        'word[:1]',
        'word[:2]',
        'word[:3]',
        'word.isupper()',
        'word.istitle()',
        'word.isdigit()',
        'postag','postag[:2]',
        'postag[:2]',
        'tamanho',
        'word.isalnum()',
        'len(word)',
        'tamanho(sent)',
        'freqwordsent',
        '-1:word',
        '-1:word.lower()',
        '-1:word.istitle()',
        '-1:word.isdigit()',
        '-1:word.isupper()',
        '-1:postag',
        '-1:postag[:2]',
        '-1:postag[:2]',
        '-1:word[-3:]',
        '-1:word[-2:]',
        '-1:word[-1:]',
        '-1:word[:1]',
        '-1:word[:2]',
        '-1:word[:3]',
        '-1:len(word)',
        '-1:word.isalnum()',
        '-2:word',
        '-2:word.lower()',
        '-2:word.istitle()',
        '-2:word.isdigit()',
        '-2:word.isupper()',
        '-2:postag',
        '-2:postag[:2]',
        '-2:word[-3:]',
        '-2:word[-2:]',
        '-2:word[-1:]',
        '-2:word[:1]',
        '-2:word[:2]',
        '-2:word[:3]',
        '-2:len(word)',
        '-2:word.isalnum()',
        '+1:word',
        '+1:word.lower()',
        '+1:word.istitle()',
        '+1:word.isdigit()',
        '+1:word.isupper()',
        '+1:postag',
        '+1:postag[:2]',
        '+1:word[-3:]',
        '+1:word[-2:]',
        '+1:word[-1:]',
        '+1:word[:1]',
        '+1:word[:2]',
        '+1:word[:3]',
        '+1:len(word)',
        '+1:word.isalnum()',
        '+2:word',
        '+2:word.lower()',
        '+2:word.istitle()',
        '+2:word.isdigit()',
        '+2:word.isupper()',
        '+2:postag',
        '+2:postag[:2]',
        '+2:word[-3:]',
        '+2:word[-2:]',
        '+2:word[-1:]',
        '+2:word[:1]',
        '+2:word[:2]',
        '+2:word[:3]',
        '+2:len(word)',
        '+2:word.isalnum()'
]

In [3]:
f"Número de atributos do modelo: {len(lista)}"

'Número de atributos do modelo: 81'

In [39]:
DIR_PL = "./dados-categorias/PLs/"  
DIR_ST = "./dados-categorias/STs/"
# DIR_PL = "./dados-tipos/PLs/"  
# DIR_ST = "./dados-tipos/STs/"

In [40]:
all_files_pl = [DIR_PL+f for f in os.listdir(DIR_PL)]
all_files_st = [DIR_ST+f for f in os.listdir(DIR_ST)]

In [41]:
def process_conll_file(location:str)->list:
    with open(location, "r") as f:
        data = f.read()
    data = data.split("\n\n")
    data = list(map(lambda x:x.split("\n"), data))
    data.pop()
    data = list(map(lambda x:[operator.itemgetter(*[0, -1])(y.split(" ")) for y in x], data))
    return data

def combine_files(locations:list)->list:
    extended = []
    for f in locations:
        extended.extend(process_conll_file(f))
    return extended

In [42]:
# Divisão entre Conjuntos de Treinamento e de Teste
# PL's
train_size_pl = int(0.75*len(all_files_pl))
random.shuffle(all_files_pl)
train_files_pl = all_files_pl[:train_size_pl]
test_files_pl = all_files_pl[train_size_pl:]
    
train_pl = combine_files(train_files_pl)
test_pl = combine_files(test_files_pl)

# ST's
all_data_st = combine_files(all_files_st)
random.shuffle(all_data_st)
train_size_st = int(0.75*len(all_data_st))
train_st = all_data_st[:train_size_st]
test_st = all_data_st[train_size_st:]

print(f"Número de Sentenças no Conjunto de Treinamento (PL): {len(train_pl)}")
print(f"Número de Sentenças no Conjunto de Teste (PL): {len(test_pl)}")
print(f"Número de Sentenças no Conjunto de Treinamento (ST): {len(train_st)}")
print(f"Número de Sentenças no Conjunto de Teste (ST): {len(test_st)}")

Número de Sentenças no Conjunto de Treinamento (PL): 7551
Número de Sentenças no Conjunto de Teste (PL): 1975
Número de Sentenças no Conjunto de Treinamento (ST): 592
Número de Sentenças no Conjunto de Teste (ST): 198


## Projetos de Lei

In [43]:
# to_store = "CRF-CV"
    
# kfold = KFold(n_splits=5)
# train = np.array(train_pl, dtype=object)
# i = 1
# for t, tt in kfold.split(train):
#     to_train = train[t].tolist()
#     to_val = train[tt].tolist()
    
#     X_train = [sent2features(s) for s in to_train]
#     y_train = [sent2labels(s) for s in to_train]

#     X_test = [sent2features(s) for s in to_val]
    
#     crf = sklearn_crfsuite.CRF(
#         algorithm='lbfgs',
#         c1=0.9,
#         c2=0.4,
#         max_iterations=100,
#         all_possible_transitions=True
#     )
#     crf.fit(X_train, y_train)
    
#     ycrf = crf.predict(X_test)
    
#     crf_file = ""
#     for preds, true in zip(ycrf, to_val):
#         for j in range(len(preds)):
#             crf_file += true[j][0] + " " + true[j][1] + " " + preds[j] + "\n"
#         crf_file += "\n"
#     with open(f"./{to_store}/predictions_file_{i}", "w") as f:
#         f.write(crf_file)
#     i += 1

In [44]:
%%time
X_train_pl = [sent2features(s) for s in train_pl]
y_train_pl = [sent2labels(s) for s in train_pl]

X_test_pl = [sent2features(s) for s in test_pl]
y_test_pl = [sent2labels(s) for s in test_pl]

CPU times: user 4min 55s, sys: 660 ms, total: 4min 55s
Wall time: 4min 56s


In [45]:
%%time
crf_pl = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=100,
    all_possible_transitions=True
)
crf_pl.fit(X_train_pl, y_train_pl)

CPU times: user 1min 22s, sys: 359 ms, total: 1min 22s
Wall time: 1min 23s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.9, c2=0.4,
    keep_tempfiles=None, max_iterations=100)

## Solicitações de Trabalho

In [46]:
%%time
X_train_st = [sent2features(s) for s in train_st]
y_train_st = [sent2labels(s) for s in train_st]

X_test_st = [sent2features(s) for s in test_st]
y_test_st = [sent2labels(s) for s in test_st]

CPU times: user 3min 2s, sys: 1.54 s, total: 3min 3s
Wall time: 3min 4s


In [47]:
%%time
crf_st = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=100,
    all_possible_transitions=True
)
crf_st.fit(X_train_st, y_train_st)

CPU times: user 57.4 s, sys: 120 ms, total: 57.5 s
Wall time: 57.5 s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.9, c2=0.4,
    keep_tempfiles=None, max_iterations=100)

## Projetos de Lei + Solicitações de Trabalho

In [48]:
%%time
X_train_plst = X_train_pl + X_train_st
y_train_plst = y_train_pl + y_train_st

X_test_plst = X_test_pl + X_test_st
y_test_plst = y_test_pl + y_test_st

crf_plst = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=100,
    all_possible_transitions=True
)
crf_plst.fit(X_train_plst, y_train_plst)

CPU times: user 2min 25s, sys: 284 ms, total: 2min 25s
Wall time: 2min 25s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.9, c2=0.4,
    keep_tempfiles=None, max_iterations=100)

## Predições

In [49]:
# CRF treinado nos projetos de lei
y_crf_pl_pl = crf_pl.predict(X_test_pl)
y_crf_pl_st = crf_pl.predict(X_test_st)
y_crf_pl_plst = crf_pl.predict(X_test_plst)

# CRF treinado nas solicitações de trabalho
y_crf_st_pl = crf_st.predict(X_test_pl)
y_crf_st_st = crf_st.predict(X_test_st)
y_crf_st_plst = crf_st.predict(X_test_plst)

# CRF treinado nos projetos de lei e solicitações de trabalho
y_crf_plst_pl = crf_plst.predict(X_test_pl)
y_crf_plst_st = crf_plst.predict(X_test_st)
y_crf_plst_plst = crf_plst.predict(X_test_plst)

In [50]:
def store_results(test, y_pred, filename):
    content = ""
    for preds, true in zip(y_pred, test):
        for j in range(len(preds)):
            content += true[j][0] + " " + true[j][1] + " " + preds[j] + "\n"
        content += "\n"
    with open(filename, "w") as f:
        f.write(content)

In [51]:
test_plst = test_pl + test_st

In [53]:
store_results(test_pl, y_crf_pl_pl, "CRF-PL-PL")
store_results(test_pl, y_crf_st_pl, "CRF-ST-PL")
store_results(test_pl, y_crf_plst_pl, "CRF-PLST-PL")

store_results(test_st, y_crf_pl_st, "CRF-PL-ST")
store_results(test_st, y_crf_st_st, "CRF-ST-ST")
store_results(test_st, y_crf_plst_st, "CRF-PLST-ST")

store_results(test_plst, y_crf_pl_plst, "CRF-PL-PLST")
store_results(test_plst, y_crf_st_plst, "CRF-ST-PLST")
store_results(test_plst, y_crf_plst_plst, "CRF-PLST-PLST")